In [ ]:
import os, sys
sys.path.append('..')
import numpy as np
# from imageio import imread
from skimage.io import imread
from skimage.transform import resize

import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
root_dir = os.path.join('/', 'mnt', 'sdb2', 'Datasets', 'asirra')
train_dir = os.path.join(root_dir, 'train')
test_dir = os.path.join(root_dir, 'test')

### Load dataset

In [ ]:
from datasets import asirra as dataset
X_trainval, y_trainval = dataset.read_asirra_subset(train_dir, one_hot=True)#, 
                                                    # sample_size=1000)    # FIXME

In [ ]:
trainval_size = X_trainval.shape[0]
val_size = int(trainval_size * 0.2)    # FIXME
# val_size = int(trainval_size * 0.5)    # FIXME
X_val, y_val = X_trainval[:val_size], y_trainval[:val_size]
X_train, y_train = X_trainval[val_size:], y_trainval[val_size:]

train_set = dataset.DataSet(X_train, y_train)
val_set = dataset.DataSet(X_val, y_val)

In [ ]:
print(X_train.shape)
print(X_train.min(), X_train.max())
print(X_val.shape)
print(X_val.min(), X_val.max())

### Visualize results of data augmentation: patch cropping, horizontal flipping

In [ ]:
batch_size = 16
batch_images, batch_labels = train_set.next_batch(batch_size, shuffle=True, 
                                                  augment=False, is_train=True)

fig, axes = plt.subplots(4, 4, figsize=(20, 20))
for j in range(4):
    for i in range(4):
        idx = 4*j+i
        axes[j, i].imshow(batch_images[idx])
        axes[j, i].set_title('label: {}'.format(batch_labels[idx]))
        _ = axes[j, i].axis('off')

In [ ]:
batch_size = 1
augment = False
batch_images, batch_labels = train_set.next_batch(batch_size, shuffle=True, 
                                                  augment=augment, is_train=False)
print(batch_images.shape)

if not augment:
    fig, ax = plt.subplots(1, 1, figsize=(5, 5))
    ax.imshow(batch_images[0])
    ax.set_title('label: {}'.format(batch_labels[0]))
    _ = ax.axis('off')
else:
    fig, axes = plt.subplots(2, 5, figsize=(25, 10))
    for j in range(2):
        for i in range(5):
            idx = 5*j+i
            axes[j, i].imshow(batch_images[0, idx])
            axes[j, i].set_title('label: {}'.format(batch_labels[0]))
            _ = axes[j, i].axis('off')

### Preliminary experiments

In [ ]:
hp_d = dict()
# Mean image
hp_d['image_mean'] = train_set.images.mean(axis=(0, 1, 2))
print(hp_d['image_mean'])
print((train_set.images - hp_d['image_mean']).mean(axis=(0, 1, 2)))

In [ ]:
# FIXME: Regularization hyperparameters
hp_d['weight_decay'] = 0.0000

# FIXME: Training hyperparameters
hp_d['batch_size'] = 256
hp_d['num_epochs'] = 320

hp_d['augment_train'] = True 
hp_d['augment_pred'] = False

hp_d['init_learning_rate'] = 0.01
hp_d['momentum'] = 0.9
hp_d['learning_rate_patience'] = 10
hp_d['learning_rate_decay'] = 0.1
hp_d['eps'] = 1e-8

# FIXME: Evaluation hyperparameters
hp_d['score_threshold'] = 1e-4

In [ ]:
import tensorflow as tf
from models.nn import AlexNet as ConvNet
from learning.optimizers import MomentumOptimizer as Optimizer
from learning.evaluators import AccuracyEvaluator as Evaluator

In [ ]:
tf.reset_default_graph()
graph = tf.get_default_graph()

config = tf.ConfigProto()
config.gpu_options.allow_growth = True

model = ConvNet([227, 227, 3], 2, **hp_d)
sess = tf.Session(graph=graph, config=config)
evaluator = Evaluator()
optimizer = Optimizer(sess, model, train_set, evaluator, **hp_d)

In [ ]:
-np.log(0.5)

In [ ]:
train_results = optimizer.train(eval_set=val_set, details=True, verbose=True, **hp_d)    # FIXME